In [25]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [26]:
df = pd.read_csv('train.csv')

In [27]:
df.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [28]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'] , inplace=True)

In [29]:
df.head()
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [30]:

# Train test split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'] , test_size=0.2, random_state=False)

In [31]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
140,3,female,NaN,0,2,15.2458,C
439,2,male,31.0,0,0,10.5000,S
817,2,male,31.0,1,1,37.0042,C
378,3,male,20.0,0,0,4.0125,C
491,3,male,21.0,0,0,7.2500,S
...,...,...,...,...,...,...,...
835,1,female,39.0,1,1,83.1583,C
192,3,female,19.0,1,0,7.8542,S
629,3,male,NaN,0,0,7.7333,Q
559,3,female,36.0,1,0,17.4000,S


In [32]:
y_train.sample(3)

20     0
177    0
857    1
Name: Survived, dtype: int64

In [33]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [34]:
# Imputations

si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age =  si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

X_test_age =  si_age.fit_transform(X_test[['Age']])
X_test_embarked = si_embarked.fit_transform(X_test[['Embarked']])

In [35]:
X_test_embarked

array([['C'],
       ['S'],
       ['Q'],
       ['C'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['C'],
       ['S'],
       ['S'],
       ['Q'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['Q'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['C'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['Q'],
       ['C'],
       ['S'],
       ['C'],
       ['C'],
       ['S'],
       ['Q'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
       ['S'],
      

In [36]:
ohe_sex = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False, handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.fit_transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.fit_transform(X_test_embarked)


In [37]:
X_train_embarked

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [38]:
# Concatenating the new transfomed arrays into one

X_train_rem = X_train.drop(columns=['Sex', 'Embarked', 'Age'], axis=1)
X_test_rem = X_test.drop(columns=['Sex', 'Embarked', 'Age'], axis=1)

In [39]:
X_test_rem.shape

(179, 4)

In [40]:
X_train_transformed = np.concatenate((X_train_rem,X_train_age,X_train_sex, X_train_embarked), axis =1)

X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_embarked, X_test_sex), axis=1)

In [41]:
X_test_transformed.shape

(179, 10)

In [42]:
clf = DecisionTreeClassifier()

In [43]:
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [44]:
pred = clf.predict(X_test_transformed)

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.6871508379888268

In [46]:
import pickle

In [47]:
pickle.dump(ohe_sex, open('model/ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('model/ohe_embaked.pkl', 'wb'))
pickle.dump(clf, open('model/clf.pkl', 'wb'))